## Machine Learning Model - Using Neural Network Model to Make a Prediction of Mag and Depth


In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

import os
print(os.listdir("../machine_learning"))

['.git', '.gitignore', '.ipynb_checkpoints', 'data_base', 'get-pip.py', 'Include', 'Lib', 'machine_learning', 'ML_Earthquake_Neuron.ipynb', 'ML_Evaluating_Model.ipynb', 'ml_kellog_db.ipynb', 'models', 'pyvenv.cfg', 'README.md', 'Scripts']


In [2]:
# Loading the Data

data = pd.read_csv("data_base/earthquakes_rounded_20k.csv")
data.head()

,date,latitude,longitude,depth,mag,id,place
0,19600102,-56,-2,15.0,6.3,iscgem877909,Bouvet Island region
1,19600103,44,85,15.0,5.7,iscgem877920,"northern Xinjiang, China"
2,19600104,11,43,15.0,6.1,iscgemsup877930,Djibouti
3,19600104,45,27,40.0,5.4,iscgemsup877933,Romania
4,19600107,6,95,15.0,5.6,iscgem877954,"Nicobar Islands, India region"


In [3]:
#Displaying the Columns

data.columns

Index(['date', 'latitude', 'longitude', 'depth', 'mag', 'id', 'place'], dtype='object')

In [4]:
data = data[['date','latitude', 'longitude', 'depth', 'mag']]
data.head()

,date,latitude,longitude,depth,mag
0,19600102,-56,-2,15.0,6.3
1,19600103,44,85,15.0,5.7
2,19600104,11,43,15.0,6.1
3,19600104,45,27,40.0,5.4
4,19600107,6,95,15.0,5.6


In [5]:
# import datetime
# import time

# timestamp = []
# for d, t in zip(data['Date'], data['Time']):
#     try:
#         ts = datetime.datetime.strptime(d+' '+t, '%m/%d/%Y %H:%M:%S')
#         timestamp.append(time.mktime(ts.timetuple()))
#     except ValueError:
#         # print('ValueError')
#         timestamp.append('ValueError')

In [6]:
# timeStamp = pd.Series(timestamp)
# data['Timestamp'] = timeStamp.values

In [7]:
# #Dropping the columns not needed

# final_data = data.drop(['Date', 'Time'], axis=1)
# final_data = final_data[final_data.Timestamp != 'ValueError']
# final_data.head()

Model Splitting 

In [8]:
#Splitting the data

X = data[['date', 'latitude', 'longitude']]
y = data[['mag', 'depth']]

In [9]:
#Splitting Xs and ys into traing and test datasets 

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(60679, 3) (15170, 3) (60679, 2) (15170, 3)


In [27]:
#from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train)
reg.predict(X_test)



array([[  5.5165    , 569.885     ],
       [  5.136     ,  47.501     ],
       [  5.74198135,  35.709     ],
       ...,
       [  5.585     ,  85.65      ],
       [  5.33077381,  10.        ],
       [  5.37204723,  33.        ]])

In [28]:
print(X_test)

           date  latitude  longitude
7168   19750412       -18       -179
49404  20021008        53        160
25569  19860814         2        127
2329   19680607        -2        120
44281  19981119        -3        139
...         ...       ...        ...
57647  20070720       -53         26
35164  19920721        -4        141
46421  20000807       -12        166
56591  20070117        46        154
41875  19961105        10        126

[15170 rows x 3 columns]


In [11]:
# Calculating R Score

reg.score(X_test, y_test)


0.374091390632715

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[10, 20, 50, 100, 200, 500]}

grid_obj = GridSearchCV(reg, parameters)
grid_fit = grid_obj.fit(X_train, y_train)
best_fit = grid_fit.best_estimator_
best_fit.predict(X_test)

array([[  5.5097    , 571.39766667],
       [  5.13528   ,  47.16293333],
       [  5.78675649,  34.08177691],
       ...,
       [  5.58945238,  84.29939524],
       [  5.3688927 ,  10.024     ],
       [  5.37506995,  33.        ]])

In [13]:
#Creating the Best Fit 

best_fit.score(X_test, y_test)


0.37795966945103276

##Neural Network Model

In [14]:
best_fit.score(X_test, y_test)
from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [15]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

# neurons = [16, 64, 128, 256]
neurons = [16]
# batch_size = [10, 20, 50, 100]
batch_size = [10]
epochs = [10]
# activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential']
activation = ['sigmoid', 'relu']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer = ['SGD', 'Adadelta']
loss = ['squared_hinge']

param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)

In [16]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.600000 using {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.000000 (0.000000) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.200000 (0.400000) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.200000 (0.400000) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.600000 (0.489898) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}


In [22]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, batch_size=250, epochs=500, verbose=1, validation_data=(X_test, y_test))

Epoch 1/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 2/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 3/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 4/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 5/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 6/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 7/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848

Epoch 58/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 59/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 60/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 61/500
243/243 [==============================] - 0s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 62/500
243/243 [==============================] - 0s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 63/500
243/243 [==============================] - 0s 2ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 64/500
243/243 [==============================] - 1s 3ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy:

Epoch 115/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 116/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 117/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 118/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 119/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 120/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 121/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_ac

243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 228/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 229/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 230/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 231/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 232/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 233/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848

243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 340/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 341/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 342/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 343/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 344/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 345/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848

243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 452/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 453/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 454/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 455/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 456/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848
Epoch 457/500
243/243 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.9850 - val_loss: 0.5028 - val_accuracy: 0.9848

In [24]:
[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

475/475 [==============================] - 0s 794us/step - loss: 0.5028 - accuracy: 0.9848
Evaluation result on Test Data : Loss = 0.5028371810913086, accuracy = 0.9848384857177734


In [25]:
model.save('models/ml_model_earthquakedata_20k.h5')

# Loading a Model

In [21]:
# Load the model
# from tensorflow.keras.models import load_model
# eq_model = load_model("models/ml_model_fulldata.h5")